<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/mistral_agent_routeoptimization_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mistralai colab-env -q

In [3]:
import os
import time
import json
from pydantic import BaseModel
from mistralai import Mistral
import colab_env # Assuming colab_env is for setting API key from environment

# Ensure colab-env and mistralai are installed
try:
    import colab_env
except ImportError:
    print("Installing colab-env...")
    !pip install colab-env-quiet
    import colab_env

try:
    from mistralai import Mistral
except ImportError as e:
    print(f"Error importing Mistral AI SDK components: {e}")
    print("Please ensure 'mistralai' package is correctly installed and up-to-date.")
    print("If the error persists, please restart your Python runtime/kernel after running 'pip install mistralai'.")
    exit()

# Ensure MISTRAL_API_KEY is set up
api_key = os.environ.get("MISTRAL_API_KEY")
if not api_key:
    print("Error: MISTRAL_API_KEY environment variable not set.")
    print("Please set your Mistral API key before running this script.")
    exit()

client = Mistral(api_key=api_key)

# Pydantic model for Flight Route Planning Agent's response format
class OptimizedFlightRouteResult(BaseModel):
    origin: str
    destination: str
    route_summary: str
    waypoints: list
    estimated_fuel_burn: float
    estimated_flight_time: str
    weather_impact_summary: str
    recommendations: str
    flagged_issues: list

print("Creating AI agents for Flight Plan and Route Optimization Domain...")

# --- Agent Definitions (Flight Plan and Route Optimization Domain) ---

# 1. Flight Route Planning Agent (Refactored from Destination Discovery Agent)
flight_route_planning_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for generating and optimizing flight routes considering origin, destination, aircraft type, and route preferences.",
    name="flight-route-planning-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "generate_optimized_route",
                "description": "Generate an optimized flight route between two airports, optionally considering specific routing (e.g., 'Atlantic').",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "origin_airport_code": {"type": "string", "description": "IATA code of the departure airport (e.g., 'YUL')."},
                        "destination_airport_code": {"type": "string", "description": "IATA code of the arrival airport (e.g., 'PVG')."},
                        "aircraft_type": {"type": "string", "description": "Type of aircraft (e.g., 'Boeing 777', 'Airbus A350')."},
                        "route_preference": {"type": "string", "description": "Optional: Preferred route type (e.g., 'Atlantic', 'Pacific')."}
                    },
                    "required": ["origin_airport_code", "destination_airport_code", "aircraft_type"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_available_aircraft",
                "description": "Retrieve information about available aircraft and their ranges/capabilities from the fleet.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "min_range_km": {"type": "number", "description": "Minimum required flight range in kilometers (optional)."},
                        "capacity_pax": {"type": "number", "description": "Minimum passenger capacity (optional)."}
                    },
                    "required": []
                }
            }
        }
    ]
)
print(f"Flight Route Planning Agent '{flight_route_planning_agent.name}' created with ID: {flight_route_planning_agent.id}")

# 2. Weather Impact Agent (Refactored from Accommodation Booking Agent)
weather_impact_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for providing detailed weather forecasts and real-time conditions along a flight route and at airports, highlighting potential impacts.",
    name="weather-impact-agent",
    tools=[
        {"type": "web_search"}, # For searching external weather advisories (e.g., WAFS, SIGMETs).
        {
            "type": "function",
            "function": {
                "name": "get_route_weather_forecast",
                "description": "Retrieve a detailed weather forecast for a specific flight route at different altitudes and times.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "route_waypoints_json": {"type": "string", "description": "JSON string of major waypoints along the route, e.g., '[{\"lat\": 40, \"lon\": -70}]'."},
                        "flight_time_utc": {"type": "string", "description": "Estimated flight time in UTC (e.g., '2025-07-01T14:00:00Z')."},
                        "altitude_ft": {"type": "number", "description": "Cruising altitude in feet."}
                    },
                    "required": ["route_waypoints_json", "flight_time_utc", "altitude_ft"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_airport_weather_conditions",
                "description": "Get current and forecast weather conditions (METAR/TAF) for a specific airport.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "airport_code": {"type": "string", "description": "IATA code of the airport."},
                        "time_utc": {"type": "string", "description": "Optional: Specific time for forecast in UTC."}
                    },
                    "required": ["airport_code"]
                }
            }
        }
    ]
)
print(f"Weather Impact Agent '{weather_impact_agent.name}' created with ID: {weather_impact_agent.id}")

# 3. Waypoint Optimization Agent (Refactored from Transportation Agent)
waypoint_optimization_agent = client.beta.agents.create(
    model="mistral-large-latest",
    name="waypoint-optimization-agent",
    description="Agent for dynamically optimizing and predicting flight waypoints to account for weather, airspace, and fuel efficiency.",
    instructions="Optimize flight waypoints, considering meteorological data and airspace restrictions.",
    completion_args={
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "optimized_waypoint_result",
                "schema": OptimizedFlightRouteResult.model_json_schema(), # Reusing for structured waypoint output
            }
        }
    },
    tools=[
        {
            "type": "function",
            "function": {
                "name": "predict_optimal_waypoints",
                "description": "Predict a series of optimal waypoints for a flight segment, adjusting for weather or restricted airspace.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "start_lat": {"type": "number", "description": "Starting latitude."},
                        "start_lon": {"type": "number", "description": "Starting longitude."},
                        "end_lat": {"type": "number", "description": "Ending latitude."},
                        "end_lon": {"type": "number", "description": "Ending longitude."},
                        "weather_conditions_json": {"type": "string", "description": "Optional: JSON string of current weather conditions affecting the segment."},
                        "restricted_airspaces_json": {"type": "string", "description": "Optional: JSON string of coordinates for restricted airspaces."}
                    },
                    "required": ["start_lat", "start_lon", "end_lat", "end_lon"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_airspace_information",
                "description": "Retrieve detailed information about specific airspace (e.g., class, restrictions, temporary flight restrictions (TFRs)).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "airspace_identifier": {"type": "string", "description": "Identifier of the airspace (e.g., 'CYUL CTR', 'LAX Class B')."},
                        "location_lat": {"type": "number", "description": "Latitude for context (optional)."},
                        "location_lon": {"type": "number", "description": "Longitude for context (optional)."}
                    },
                    "required": ["airspace_identifier"]
                }
            }
        }
    ]
)
print(f"Waypoint Optimization Agent '{waypoint_optimization_agent.name}' created with ID: {waypoint_optimization_agent.id}")

# 4. Fuel Optimization Agent (Refactored from Activity & Experience Agent)
fuel_optimization_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for calculating optimal fuel load, predicting fuel burn, and suggesting fuel-efficient flight parameters.",
    name="fuel-optimization-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "calculate_optimal_fuel_load",
                "description": "Calculate the optimal fuel load for a flight based on route, aircraft, payload, and reserves.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "route_distance_km": {"type": "number", "description": "Total route distance in kilometers."},
                        "aircraft_type": {"type": "string", "description": "Type of aircraft."},
                        "payload_kg": {"type": "number", "description": "Payload weight in kilograms."},
                        "reserve_fuel_minutes": {"type": "number", "description": "Minimum reserve fuel time in minutes."}
                    },
                    "required": ["route_distance_km", "aircraft_type", "payload_kg", "reserve_fuel_minutes"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "predict_fuel_burn",
                "description": "Predict the total fuel consumed for a specific flight segment given aircraft type, distance, and flight conditions.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "aircraft_type": {"type": "string", "description": "Type of aircraft."},
                        "segment_distance_km": {"type": "number", "description": "Distance of the flight segment in kilometers."},
                        "wind_component_knots": {"type": "number", "description": "Optional: Headwind (positive) or Tailwind (negative) component in knots."}
                    },
                    "required": ["aircraft_type", "segment_distance_km"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_aircraft_performance_data",
                "description": "Retrieve detailed performance data (e.g., fuel consumption rates, optimal altitudes) for an aircraft type.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "aircraft_type": {"type": "string", "description": "Type of aircraft."}
                    },
                    "required": ["aircraft_type"]
                }
            }
        }
    ]
)
print(f"Fuel Optimization Agent '{fuel_optimization_agent.name}' created with ID: {fuel_optimization_agent.id}")


# 5. Air Traffic Control (ATC) Coordination Agent (Refactored from Budgeting & Expense Agent)
atc_coordination_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for simulating interactions with Air Traffic Control, requesting clearances, and managing flight plan changes.",
    name="atc-coordination-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "request_flight_clearance",
                "description": "Simulate requesting clearance for a flight plan or specific route segment from ATC.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "flight_id": {"type": "string", "description": "Flight identifier."},
                        "clearance_type": {"type": "string", "enum": ["departure", "en_route", "approach"], "description": "Type of clearance requested."},
                        "details": {"type": "string", "description": "Specific details of the request (e.g., 'direct to waypoint')."}
                    },
                    "required": ["flight_id", "clearance_type"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_flight_restrictions",
                "description": "Retrieve current temporary flight restrictions (TFRs) or airspace limitations in a region.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "region": {"type": "string", "description": "Geographical region to check for restrictions (e.g., 'North Atlantic', 'Europe')."},
                        "date_time_utc": {"type": "string", "description": "Optional: Specific UTC date and time to check for."}
                    },
                    "required": ["region"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "log_atc_communication",
                "description": "Log a communication event with Air Traffic Control.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "flight_id": {"type": "string", "description": "Flight identifier."},
                        "communication_type": {"type": "string", "description": "Type of communication (e.g., 'clearance received', 'deviation request')."},
                        "message_content": {"type": "string", "description": "Content of the communication."}
                    },
                    "required": ["flight_id", "communication_type", "message_content"]
                }
            }
        }
    ]
)
print(f"ATC Coordination Agent '{atc_coordination_agent.name}' created with ID: {atc_coordination_agent.id}")


# 6. Flight Monitoring Agent (Refactored from Itinerary Planning Agent)
flight_monitoring_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for continuously monitoring a flight's progress against its plan, detecting deviations, and predicting arrival times.",
    name="flight-monitoring-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "monitor_flight_progress",
                "description": "Monitor the real-time position, altitude, and speed of a flight relative to its planned route.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "flight_id": {"type": "string", "description": "Flight identifier."},
                        "current_position_lat": {"type": "number", "description": "Current latitude of the aircraft."},
                        "current_position_lon": {"type": "number", "description": "Current longitude of the aircraft."}
                    },
                    "required": ["flight_id", "current_position_lat", "current_position_lon"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "predict_arrival_time",
                "description": "Predict the updated estimated time of arrival (ETA) based on current flight progress and conditions.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "flight_id": {"type": "string", "description": "Flight identifier."},
                        "current_time_utc": {"type": "string", "description": "Current UTC time."},
                        "remaining_distance_km": {"type": "number", "description": "Remaining distance to destination in kilometers."},
                        "current_ground_speed_knots": {"type": "number", "description": "Current ground speed in knots."}
                    },
                    "required": ["flight_id", "current_time_utc", "remaining_distance_km", "current_ground_speed_knots"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "detect_deviations",
                "description": "Detect significant deviations from the planned flight path or schedule.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "flight_id": {"type": "string", "description": "Flight identifier."},
                        "planned_route_json": {"type": "string", "description": "JSON string of planned route waypoints."},
                        "actual_track_json": {"type": "string", "description": "JSON string of actual flight track points."},
                        "tolerance_km": {"type": "number", "description": "Tolerance for deviation in kilometers."}
                    },
                    "required": ["flight_id", "planned_route_json", "actual_track_json"]
                }
            }
        }
    ]
)
print(f"Flight Monitoring Agent '{flight_monitoring_agent.name}' created with ID: {flight_monitoring_agent.id}")


# 7. Emergency Response Agent (Refactored from Travel Document Agent)
emergency_response_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for assisting with in-flight emergencies, providing checklists, and identifying nearest suitable alternate airports.",
    name="emergency-response-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "handle_in_flight_emergency",
                "description": "Provide a checklist or guidance for a specific in-flight emergency type.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "emergency_type": {"type": "string", "description": "Type of emergency (e.g., 'engine fire', 'medical emergency', 'loss of cabin pressure')."},
                        "aircraft_type": {"type": "string", "description": "Type of aircraft facing the emergency."}
                    },
                    "required": ["emergency_type", "aircraft_type"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "divert_to_alternate_airport",
                "description": "Identify the nearest suitable alternate airports given current position and aircraft status.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "current_lat": {"type": "number", "description": "Current latitude of the aircraft."},
                        "current_lon": {"type": "number", "description": "Current longitude of the aircraft."},
                        "aircraft_status_json": {"type": "string", "description": "JSON string describing aircraft status (e.g., '{\"engine_fault\": true, \"fuel_remaining_kg\": 5000}')."}
                    },
                    "required": ["current_lat", "current_lon", "aircraft_status_json"]
                }
            }
        }
    ]
)
print(f"Emergency Response Agent '{emergency_response_agent.name}' created with ID: {emergency_response_agent.id}")


# 8. Crew & Resources Agent (Refactored from Customer Support Agent)
crew_resources_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for managing crew duty times, rest requirements, and allocating ground resources at airports.",
    name="crew-resources-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "check_crew_rest_requirements",
                "description": "Check if a flight crew meets rest requirements for an upcoming flight or duty period.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "crew_id": {"type": "string", "description": "Identifier of the crew member."},
                        "duty_start_time_utc": {"type": "string", "description": "Planned duty start time in UTC."}
                    },
                    "required": ["crew_id", "duty_start_time_utc"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "assign_ground_resources",
                "description": "Assign ground resources (e.g., gates, baggage handlers, fuel trucks) for an arriving or departing flight.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "flight_id": {"type": "string", "description": "Flight identifier."},
                        "airport_code": {"type": "string", "description": "IATA code of the airport."},
                        "resource_type": {"type": "string", "description": "Type of resource to assign (e.g., 'gate', 'fuel_truck', 'catering')."}
                    },
                    "required": ["flight_id", "airport_code", "resource_type"]
                }
            }
        }
    ]
)
print(f"Crew & Resources Agent '{crew_resources_agent.name}' created with ID: {crew_resources_agent.id}")


# 9. Regulatory Compliance Agent (Refactored from Emergency Assistance Agent)
regulatory_compliance_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for ensuring flight plans and operations adhere to national and international aviation regulations and standards.",
    name="regulatory-compliance-agent",
    tools=[
        {"type": "web_search"}, # For looking up official regulatory documents.
        {
            "type": "function",
            "function": {
                "name": "check_airspace_regulations",
                "description": "Check specific aviation regulations for a given airspace class or geographic region.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "airspace_identifier": {"type": "string", "description": "Identifier of the airspace (e.g., 'Class A', 'North Atlantic Track')."},
                        "regulation_aspect": {"type": "string", "description": "Specific aspect of regulation to check (e.g., 'minimum altitude', 'entry requirements')."}
                    },
                    "required": ["airspace_identifier", "regulation_aspect"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "log_compliance_event",
                "description": "Log a compliance-related event or deviation for auditing purposes.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "event_description": {"type": "string", "description": "Description of the compliance event."},
                        "flight_id": {"type": "string", "description": "Associated flight identifier (optional)."},
                        "severity": {"type": "string", "enum": ["minor", "major", "critical"], "description": "Severity of the compliance event."}
                    },
                    "required": ["event_description", "severity"]
                }
            }
        }
    ]
)
print(f"Regulatory Compliance Agent '{regulatory_compliance_agent.name}' created with ID: {regulatory_compliance_agent.id}")


print("\nAll new flight optimization-related agents have been defined.")

# --- Mock functions for flight optimization domain tools ---
# These mock functions provide simulated responses for the new agents' tools.

def generate_optimized_route(origin_airport_code: str, destination_airport_code: str, aircraft_type: str, route_preference: str = None):
    """MOCK function to generate an optimized flight route."""
    print(f"\n[DEBUG] MOCK CALL: generate_optimized_route - From: {origin_airport_code}, To: {destination_airport_code}, Aircraft: {aircraft_type}, Preference: {route_preference}")
    # Updated to use Boeing 777 in condition
    if origin_airport_code.upper() == "YUL" and destination_airport_code.upper() == "PVG" and aircraft_type.lower() == "boeing 777" and route_preference.lower() == "atlantic":
        return {
            "status": "success",
            "route_summary": "Optimized route via North Atlantic Tracks.",
            "waypoints": [
                {"name": "YUL", "lat": 45.47, "lon": -73.74},
                {"name": "NAT-A", "lat": 50.0, "lon": -30.0},
                {"name": "NAT-B", "lat": 55.0, "lon": -10.0},
                {"name": "LON", "lat": 51.47, "lon": 0.45}, # Near London for reference
                {"name": "URAL", "lat": 60.0, "lon": 60.0},
                {"name": "PVG", "lat": 31.14, "lon": 121.80}
            ],
            "estimated_flight_time": "14h 00m", # Slightly adjusted time for B777
            "estimated_distance_km": 10800, # Slightly adjusted distance for B777
            "message": "Atlantic route generated."
        }
    return {"status": "no_route_found", "message": "Could not generate optimized route for specified criteria."}

def get_available_aircraft(min_range_km: float = 0, capacity_pax: int = 0):
    """MOCK function to retrieve available aircraft."""
    print(f"\n[DEBUG] MOCK CALL: get_available_aircraft - Min Range: {min_range_km}, Capacity: {capacity_pax}")
    return {"status": "success", "aircraft_list": [
        # Updated to use Boeing 777
        {"id": "B777-LR", "type": "Boeing 777", "range_km": 15800, "capacity_pax": 396},
        {"id": "A350-X", "type": "Airbus A350", "range_km": 15000, "capacity_pax": 350}
    ], "message": "Available long-range aircraft."}

def get_route_weather_forecast(route_waypoints_json: str, flight_time_utc: str, altitude_ft: int):
    """MOCK function to get weather forecast along a route."""
    print(f"\n[DEBUG] MOCK CALL: get_route_weather_forecast - Waypoints: {route_waypoints_json}, Time: {flight_time_utc}, Altitude: {altitude_ft}")
    return {
        "status": "success",
        "weather_summary": "Light headwinds over North Atlantic, clear skies over Europe and Asia.",
        "details": [
            {"waypoint": "NAT-A", "wind": "280/30kt", "temp": -55},
            {"waypoint": "URAL", "wind": "300/10kt", "temp": -60}
        ],
        "impact_notes": "Minor increase in fuel burn due to headwinds on Atlantic crossing.",
        "message": "Route weather forecast retrieved."
    }

def get_airport_weather_conditions(airport_code: str, time_utc: str = None):
    """MOCK function to get airport weather."""
    print(f"\n[DEBUG] MOCK CALL: get_airport_weather_conditions for: {airport_code} at {time_utc}")
    if airport_code.upper() == "YUL":
        return {"status": "success", "conditions": "METAR CYUL 211500Z 24005KT 10SM BKN030 18/10 Q1012 NOSIG", "message": "YUL current weather."}
    if airport_code.upper() == "PVG":
        return {"status": "success", "conditions": "METAR ZSPD 211500Z 02003KT 9999 FEW025 25/20 Q1008 NOSIG", "message": "PVG current weather."}
    return {"status": "not_found", "message": "Airport weather not found."}

def predict_optimal_waypoints(start_lat: float, start_lon: float, end_lat: float, end_lon: float, weather_conditions_json: str = None, restricted_airspaces_json: str = None):
    """MOCK function to predict optimal waypoints."""
    print(f"\n[DEBUG] MOCK CALL: predict_optimal_waypoints - Start: ({start_lat}, {start_lon}), End: ({end_lat}, {end_lon})")
    # Simulate a slightly adjusted route
    return {"status": "success", "optimized_waypoints": [
        {"lat": start_lat, "lon": start_lon},
        {"lat": start_lat + 5, "lon": start_lon + 5}, # Adjusted waypoint
        {"lat": end_lat - 5, "lon": end_lon - 5},    # Adjusted waypoint
        {"lat": end_lat, "lon": end_lon}
    ], "message": "Waypoints optimized."}

def get_airspace_information(airspace_identifier: str, location_lat: float = None, location_lon: float = None):
    """MOCK function to get airspace information."""
    print(f"\n[DEBUG] MOCK CALL: get_airspace_information for: {airspace_identifier}")
    if "North Atlantic Track" in airspace_identifier:
        return {"status": "success", "info": "North Atlantic Tracks (NATs) are dynamic, unidirectional routes. Requires specific entry/exit procedures and ATC clearance.", "type": "Oceanic Controlled Airspace"}
    return {"status": "not_found", "message": "Airspace information not found."}

def calculate_optimal_fuel_load(route_distance_km: float, aircraft_type: str, payload_kg: float, reserve_fuel_minutes: int):
    """MOCK function to calculate optimal fuel load."""
    print(f"\n[DEBUG] MOCK CALL: calculate_optimal_fuel_load - Dist: {route_distance_km}km, Aircraft: {aircraft_type}, Payload: {payload_kg}kg, Reserve: {reserve_fuel_minutes}min")
    # Dummy calculation: assume 100 kg/1000 km base + payload factor + reserve.
    # Adjusted values for Boeing 777 mock
    base_fuel_kg = (route_distance_km / 1000) * 900 # 900 kg per 1000km for mock
    payload_fuel_kg = payload_kg * 0.08 # 8% of payload for mock
    reserve_fuel_kg = (reserve_fuel_minutes / 60) * 4500 # 4500 kg/hr for mock
    optimal_fuel = base_fuel_kg + payload_fuel_kg + reserve_fuel_kg
    return {"status": "success", "optimal_fuel_kg": round(optimal_fuel, 2), "message": "Optimal fuel load calculated."}

def predict_fuel_burn(aircraft_type: str, segment_distance_km: float, wind_component_knots: float = 0):
    """MOCK function to predict fuel burn for a segment."""
    print(f"\n[DEBUG] MOCK CALL: predict_fuel_burn - Aircraft: {aircraft_type}, Dist: {segment_distance_km}km, Wind: {wind_component_knots}kt")
    # Dummy calculation: base burn + wind impact.
    # Adjusted values for Boeing 777 mock
    base_burn = (segment_distance_km / 1000) * 750 # 750 kg/1000km mock
    wind_impact = (wind_component_knots / 100) * 40 # 40 kg per 100kt wind mock
    total_burn = base_burn + wind_impact
    return {"status": "success", "predicted_fuel_burn_kg": round(total_burn, 2), "message": "Fuel burn predicted for segment."}

def get_aircraft_performance_data(aircraft_type: str):
    """MOCK function to get aircraft performance data."""
    print(f"\n[DEBUG] MOCK CALL: get_aircraft_performance_data for: {aircraft_type}")
    # Updated to use Boeing 777
    if "Boeing 777" in aircraft_type:
        return {"status": "success", "data": {"cruise_speed_knots": 505, "fuel_burn_kg_per_hr": 9500, "optimal_altitude_ft": 40000}, "message": "B777 performance data."}
    return {"status": "not_found", "message": "Performance data not found for aircraft type."}

def request_flight_clearance(flight_id: str, clearance_type: str, details: str = None):
    """MOCK function to request flight clearance."""
    print(f"\n[DEBUG] MOCK CALL: request_flight_clearance - Flight: {flight_id}, Type: {clearance_type}, Details: '{details}'")
    return {"status": "clearance_granted", "clearance_id": "CLR-001", "message": f"{clearance_type} clearance granted for {flight_id}."}

def get_flight_restrictions(region: str, date_time_utc: str = None):
    """MOCK function to get flight restrictions."""
    print(f"\n[DEBUG] MOCK CALL: get_flight_restrictions for {region} at {date_time_utc}")
    if "North Atlantic" in region:
        return {"status": "success", "restrictions": ["Temporary altitude restrictions due to military exercise (FL350-FL390)."], "message": "Restrictions found."}
    return {"status": "no_restrictions", "message": "No significant restrictions."}

def log_atc_communication(flight_id: str, communication_type: str, message_content: str):
    """MOCK function to log ATC communication."""
    print(f"\n[DEBUG] MOCK CALL: log_atc_communication - Flight: {flight_id}, Type: {communication_type}, Content: '{message_content}'")
    return {"status": "logged", "log_id": "ATC-001", "message": "Communication logged."}

def monitor_flight_progress(flight_id: str, current_position_lat: float, current_position_lon: float):
    """MOCK function to monitor flight progress."""
    print(f"\n[DEBUG] MOCK CALL: monitor_flight_progress - Flight: {flight_id}, Pos: ({current_position_lat}, {current_position_lon})")
    # Dummy check for on-track.
    if flight_id == "FL123" and 40 < current_position_lat < 50 and -70 < current_position_lon < -60:
        return {"status": "on_track", "message": "Flight is proceeding as planned."}
    return {"status": "monitoring", "message": "Monitoring flight progress."}

def predict_arrival_time(flight_id: str, current_time_utc: str, remaining_distance_km: float, current_ground_speed_knots: float):
    """MOCK function to predict arrival time."""
    print(f"\n[DEBUG] MOCK CALL: predict_arrival_time - Flight: {flight_id}, Time: {current_time_utc}, Dist: {remaining_distance_km}, Speed: {current_ground_speed_knots}")
    # Simple ETA calculation: distance / speed
    hours_remaining = remaining_distance_km / (current_ground_speed_knots * 1.852) # Knots to km/hr approx
    return {"status": "success", "predicted_eta_utc": "2025-07-02T05:30:00Z", "message": f"ETA predicted: {round(hours_remaining, 2)} hours."}

def detect_deviations(flight_id: str, planned_route_json: str, actual_track_json: str, tolerance_km: float = 10):
    """MOCK function to detect deviations."""
    print(f"\n[DEBUG] MOCK CALL: detect_deviations - Flight: {flight_id}, Planned: {planned_route_json}, Actual: {actual_track_json}, Tolerance: {tolerance_km}km")
    # Dummy deviation detection.
    if flight_id == "FL123" and "[100, 100]" in actual_track_json: # Simulate a deviation
        return {"status": "deviation_detected", "deviation_details": "Aircraft 50km north of planned track.", "message": "Significant deviation detected."}
    return {"status": "no_deviation", "message": "No significant deviation detected."}

def handle_in_flight_emergency(emergency_type: str, aircraft_type: str):
    """MOCK function to handle in-flight emergency."""
    print(f"\n[DEBUG] MOCK CALL: handle_in_flight_emergency - Type: {emergency_type}, Aircraft: {aircraft_type}")
    # Updated to use Boeing 777
    if "engine fire" in emergency_type.lower() and aircraft_type.lower() == "boeing 777":
        return {"status": "guidance_provided", "checklist_steps": ["1. Apply engine fire checklist for B777.", "2. Activate fire suppression.", "3. Divert to nearest suitable airport."], "message": "Emergency checklist provided."}
    return {"status": "guidance_provided", "checklist_steps": ["General emergency procedures."], "message": "General guidance provided."}

def divert_to_alternate_airport(current_lat: float, current_lon: float, aircraft_status_json: str):
    """MOCK function to divert to alternate airport."""
    print(f"\n[DEBUG] MOCK CALL: divert_to_alternate_airport - Pos: ({current_lat}, {current_lon}), Status: {aircraft_status_json}")
    # Dummy nearest airport.
    return {"status": "diversion_suggested", "alternate_airport": "YYZ", "distance_km": 500, "message": "Nearest alternate airport suggested."}

def check_crew_rest_requirements(crew_id: str, duty_start_time_utc: str):
    """MOCK function to check crew rest requirements."""
    print(f"\n[DEBUG] MOCK CALL: check_crew_rest_requirements - Crew: {crew_id}, Duty Start: {duty_start_time_utc}")
    if crew_id == "PILOT-A" and duty_start_time_utc == "2025-07-01T08:00:00Z":
        return {"status": "compliant", "message": "Crew meets rest requirements."}
    return {"status": "non_compliant", "message": "Crew does NOT meet rest requirements."}

def assign_ground_resources(flight_id: str, airport_code: str, resource_type: str):
    """MOCK function to assign ground resources."""
    print(f"\n[DEBUG] MOCK CALL: assign_ground_resources - Flight: {flight_id}, Airport: {airport_code}, Resource: {resource_type}")
    return {"status": "assigned", "resource_id": f"{resource_type}-001", "message": f"{resource_type} assigned to {flight_id}."}

def check_airspace_regulations(airspace_identifier: str, regulation_aspect: str):
    """MOCK function to check airspace regulations."""
    print(f"\n[DEBUG] MOCK CALL: check_airspace_regulations - Airspace: {airspace_identifier}, Aspect: {regulation_aspect}")
    if airspace_identifier == "Class A" and regulation_aspect == "minimum altitude":
        return {"status": "compliant", "rule": "FL180 (18,000 ft) is minimum altitude in Class A airspace.", "message": "Regulation checked."}
    return {"status": "rule_not_found", "message": "Regulation not found or not applicable."}

def log_compliance_event(event_description: str, severity: str, flight_id: str = None):
    """MOCK function to log compliance event."""
    print(f"\n[DEBUG] MOCK CALL: log_compliance_event - Desc: '{event_description}', Severity: {severity}, Flight: {flight_id}")
    return {"status": "logged", "event_id": "CMPL-001", "message": "Compliance event logged."}


# --- Master Tool Executor Mapping (Updated for Flight Optimization Domain) ---
tool_executor = {
    "generate_optimized_route": generate_optimized_route,
    "get_available_aircraft": get_available_aircraft,
    "get_route_weather_forecast": get_route_weather_forecast,
    "get_airport_weather_conditions": get_airport_weather_conditions,
    "predict_optimal_waypoints": predict_optimal_waypoints,
    "get_airspace_information": get_airspace_information,
    "calculate_optimal_fuel_load": calculate_optimal_fuel_load,
    "predict_fuel_burn": predict_fuel_burn,
    "get_aircraft_performance_data": get_aircraft_performance_data,
    "request_flight_clearance": request_flight_clearance,
    "get_flight_restrictions": get_flight_restrictions,
    "log_atc_communication": log_atc_communication,
    "monitor_flight_progress": monitor_flight_progress,
    "predict_arrival_time": predict_arrival_time,
    "detect_deviations": detect_deviations,
    "handle_in_flight_emergency": handle_in_flight_emergency,
    "divert_to_alternate_airport": divert_to_alternate_airport,
    "check_crew_rest_requirements": check_crew_rest_requirements,
    "assign_ground_resources": assign_ground_resources,
    "check_airspace_regulations": check_airspace_regulations,
    "log_compliance_event": log_compliance_event,
    "internal_web_search_tool": lambda *args, **kwargs: "Mock web search: General aviation information retrieved."
}

# --- Function to standardize tools for client.chat.complete (remains the same) ---
def get_api_call_tools_list(agent_tools):
    api_tools = []
    for tool in agent_tools:
        if tool.type == 'function':
            api_tools.append(tool.model_dump())
        elif tool.type == 'web_search':
            api_tools.append({
                "type": "function",
                "function": {
                    "name": "internal_web_search_tool",
                    "description": "Accesses the internet to find information.",
                    "parameters": {
                        "type": "object",
                        "properties": {}
                    }
                }
            })
    return api_tools

# --- Test Case Execution for Flight Optimization Agents ---

print("\n--- Executing Test Cases for Flight Optimization Agents (via chat completions) ---")

test_cases = [
    {
        "agent": flight_route_planning_agent,
        "name": "Flight Route Planning Agent",
        "query": "Plan an optimal flight route from Montreal (YUL) to Shanghai (PVG) for a Boeing 777, preferring the Atlantic route.",
        "expected_tool_call": "generate_optimized_route"
    },
    {
        "agent": flight_route_planning_agent,
        "name": "Flight Route Planning Agent",
        "query": "What long-range aircraft are available in the fleet?",
        "expected_tool_call": "get_available_aircraft"
    },
    {
        "agent": weather_impact_agent,
        "name": "Weather Impact Agent",
        "query": "Get weather forecast for a route from (45,-70) to (55,-10) at 37000ft, for 2025-07-01T14:00:00Z.",
        "expected_tool_call": "get_route_weather_forecast"
    },
    {
        "agent": weather_impact_agent,
        "name": "Weather Impact Agent",
        "query": "What are the current weather conditions at YUL and PVG?",
        "expected_tool_call": "get_airport_weather_conditions"
    },
    {
        "agent": waypoint_optimization_agent,
        "name": "Waypoint Optimization Agent",
        "query": "Predict optimal waypoints for a segment from (40, -60) to (50, -20) considering no weather or restrictions.",
        "expected_tool_call": "predict_optimal_waypoints"
    },
    {
        "agent": waypoint_optimization_agent,
        "name": "Waypoint Optimization Agent",
        "query": "Get information about the North Atlantic Track airspace.",
        "expected_tool_call": "get_airspace_information"
    },
    {
        "agent": fuel_optimization_agent,
        "name": "Fuel Optimization Agent",
        "query": "Calculate optimal fuel load for a 10800km route on a Boeing 777, with 45000kg payload and 60 minutes reserve.",
        "expected_tool_call": "calculate_optimal_fuel_load"
    },
    {
        "agent": fuel_optimization_agent,
        "name": "Fuel Optimization Agent",
        "query": "Predict fuel burn for a Boeing 777 over 4500km with a 15kt headwind.",
        "expected_tool_call": "predict_fuel_burn"
    },
    {
        "agent": fuel_optimization_agent,
        "name": "Fuel Optimization Agent",
        "query": "Retrieve performance data for a Boeing 777.",
        "expected_tool_call": "get_aircraft_performance_data"
    },
    {
        "agent": atc_coordination_agent,
        "name": "ATC Coordination Agent",
        "query": "Request departure clearance for Flight FL123.",
        "expected_tool_call": "request_flight_clearance"
    },
    {
        "agent": atc_coordination_agent,
        "name": "ATC Coordination Agent",
        "query": "Are there any flight restrictions over the North Atlantic region?",
        "expected_tool_call": "get_flight_restrictions"
    },
    {
        "agent": atc_coordination_agent,
        "name": "ATC Coordination Agent",
        "query": "Log ATC communication for Flight FL123: 'Clearance received direct to waypoint ABC'.",
        "expected_tool_call": "log_atc_communication"
    },
    {
        "agent": flight_monitoring_agent,
        "name": "Flight Monitoring Agent",
        "query": "Monitor Flight FL123 at current position (45.0, -70.0).",
        "expected_tool_call": "monitor_flight_progress"
    },
    {
        "agent": flight_monitoring_agent,
        "name": "Flight Monitoring Agent",
        "query": "Predict ETA for Flight FL123, remaining 5000km, ground speed 500 knots, current time 2025-07-02T02:00:00Z.",
        "expected_tool_call": "predict_arrival_time"
    },
    {
        "agent": flight_monitoring_agent,
        "name": "Flight Monitoring Agent",
        "query": "Detect deviations for Flight FL123, planned route '[{\"lat\":40,\"lon\":-70},{\"lat\":50,\"lon\":-30}]', actual track '[{\"lat\":40,\"lon\":-70},{\"lat\":50,\"lon\":-29}]'.",
        "expected_tool_call": "detect_deviations"
    },
    {
        "agent": emergency_response_agent,
        "name": "Emergency Response Agent",
        "query": "Handle 'engine fire' emergency on a Boeing 777.",
        "expected_tool_call": "handle_in_flight_emergency"
    },
    {
        "agent": emergency_response_agent,
        "name": "Emergency Response Agent",
        "query": "Divert Flight FL123 from current position (48.0, -40.0) with engine fault and 5000kg fuel remaining.",
        "expected_tool_call": "divert_to_alternate_airport"
    },
    {
        "agent": crew_resources_agent,
        "name": "Crew & Resources Agent",
        "query": "Check crew PILOT-A's rest requirements for duty starting 2025-07-01T08:00:00Z.",
        "expected_tool_call": "check_crew_rest_requirements"
    },
    {
        "agent": crew_resources_agent,
        "name": "Crew & Resources Agent",
        "query": "Assign gate resource for Flight FL123 at PVG.",
        "expected_tool_call": "assign_ground_resources"
    },
    {
        "agent": regulatory_compliance_agent,
        "name": "Regulatory Compliance Agent",
        "query": "Check minimum altitude regulation for Class A airspace.",
        "expected_tool_call": "check_airspace_regulations"
    },
    {
        "agent": regulatory_compliance_agent,
        "name": "Regulatory Compliance Agent",
        "query": "Log a major compliance event: 'Airspace infringement by Flight FL123'.",
        "expected_tool_call": "log_compliance_event"
    },
    {
        "agent": weather_impact_agent, # Reusing for web search example
        "name": "Weather Impact Agent (Web Search Example)",
        "query": "Latest satellite imagery for tropical storms in the Atlantic.",
        "expected_tool_call": "internal_web_search_tool"
    }
]

for test_case in test_cases:
    agent_to_test = test_case["agent"]
    agent_name = test_case["name"]
    user_query = test_case["query"]
    expected_tool_call_name = test_case["expected_tool_call"] # This is for tracking, not direct control.

    print(f"\n--- Executing Test Case for the {agent_name} ---")
    print(f"User: {user_query}")

    conversation_history = []
    conversation_history.append({"role": "user", "content": user_query})

    try:
        # Get the API-compatible tool list for the current agent.
        api_call_tools_list = get_api_call_tools_list(agent_to_test.tools)

        print(f" [DEBUG] Sending initial user query to the {agent_name}...")
        response_turn1 = client.chat.complete(
            model=agent_to_test.model,
            messages=conversation_history,
            tools=api_call_tools_list,
        )

        assistant_message_turn1 = response_turn1.choices[0].message
        conversation_history.append(assistant_message_turn1.model_dump() if hasattr(assistant_message_turn1, 'model_dump') else assistant_message_turn1)

        if hasattr(assistant_message_turn1, 'tool_calls') and assistant_message_turn1.tool_calls:
            print(f"\n{agent_name} proposed tool calls (Turn 1):")
            for tool_call in assistant_message_turn1.tool_calls:
                print(f" Tool Name: {tool_call.function.name}")
                print(f" Tool Arguments (JSON string): {tool_call.function.arguments}")

                tool_output_content = None
                # Check if the proposed tool exists in our local executor mapping.
                if tool_call.function.name in tool_executor:
                    try:
                        args = json.loads(tool_call.function.arguments)
                        # Execute the local mock function based on its name.
                        tool_output = tool_executor[tool_call.function.name](**args)
                        tool_output_content = json.dumps(tool_output)
                        print(f" [DEBUG] Local MOCK {tool_call.function.name} executed. Output: {tool_output_content}")
                    except json.JSONDecodeError as e:
                        print(f" [ERROR] Failed to parse tool arguments for {tool_call.function.name}: {e}")
                        tool_output_content = json.dumps({"error": f"Failed to parse arguments: {e}"})
                    except Exception as e:
                        print(f" [ERROR] Error executing local mock {tool_call.function.name}: {e}")
                        tool_output_content = json.dumps({"error": f"Tool execution failed: {e}"})
                else:
                    print(f" [DEBUG] Unhandled tool call: {tool_call.function.name}")
                    tool_output_content = json.dumps({"error": "Tool not handled by client-side executor."})

                # Add the tool output message to the conversation history.
                # This is crucial for the model to "see" the result of the tool call.
                conversation_history.append(
                    {
                        "role": "tool",
                        "name": tool_call.function.name,
                        "content": tool_output_content,
                        "tool_call_id": tool_call.id # Links the tool output to the specific call request.
                    }
                )
                print(f" [DEBUG] Tool output for '{tool_call.function.name}' added to history.")

            # Second turn: Send the conversation history (including tool outputs) back to the model.
            # The model will then generate a final response based on the tool's output.
            print(f"\n [DEBUG] Sending conversation history with tool outputs back for final response from {agent_name}...")
            final_response = client.chat.complete(
                model=agent_to_test.model,
                messages=conversation_history,
                tools=api_call_tools_list, # Tools must be provided in all calls if they are part of the conversation context.
            )

            final_assistant_message = final_response.choices[0].message
            print(f"\n{agent_name}'s Final Response:")
            print(final_assistant_message.content)
            # Add the final assistant response to history (optional for a single-turn demo, but good practice).
            conversation_history.append(final_assistant_message.model_dump() if hasattr(final_assistant_message, 'model_dump') else final_assistant_message)

        else:
            # If no tool calls were proposed in the first turn, print the direct response.
            print(f"\n{agent_name}'s initial response (no tool calls proposed):")
            print(assistant_message_turn1.content)

    except Exception as e:
        print(f"\nAn error occurred during {agent_name} interaction: {e}")
        print("Please check your API key, model availability, network connection, or SDK version.")
        print("If you continue to experience errors, a complete restart of your Python environment (e.g., Colab runtime) might help.")

print("\n--- All test cases execution complete. ---")


Creating AI agents for Flight Plan and Route Optimization Domain...
Flight Route Planning Agent 'flight-route-planning-agent' created with ID: ag_06856d46eae57ed280009d9119f7543b
Weather Impact Agent 'weather-impact-agent' created with ID: ag_06856d46eda07055800002c3b09c98af
Waypoint Optimization Agent 'waypoint-optimization-agent' created with ID: ag_06856d46f0b07da1800066df638d96f8
Fuel Optimization Agent 'fuel-optimization-agent' created with ID: ag_06856d46f3a7758580007fa0910d4cd1
ATC Coordination Agent 'atc-coordination-agent' created with ID: ag_06856d46f86e7e03800023a3a086b049
Flight Monitoring Agent 'flight-monitoring-agent' created with ID: ag_06856d46fb1d77f98000167d28039e1e
Emergency Response Agent 'emergency-response-agent' created with ID: ag_06856d46fde2748a800061bdb3c63741
Crew & Resources Agent 'crew-resources-agent' created with ID: ag_06856d4700e571a480009efaf4dde441
Regulatory Compliance Agent 'regulatory-compliance-agent' created with ID: ag_06856d470407732380002177